In [9]:
# taken partially from http://greenteapress.com/wp/semaphores/
import threading
import numpy as np

In [10]:
class Shared:
    def __init__(self, topcount=10):
        self.counter = 0
        self.topcount = topcount
        self.histo = np.zeros(topcount, dtype = np.int32)
        
def child(shared):
    while True:
        if shared.counter >= shared.topcount: break
        shared.histo[shared.counter] += 1
        shared.counter += 1

Sharing a single counter for two threads
------------------

- An array of as many entries as the count *topcount* value is created with 0 as initial values

- On each count the corresponding entry to the count is incremented, so if everything is OK all entries should be 1

In [11]:
myshare = Shared(10)
child1 = threading.Thread(target=child, args=(myshare,))
child2 = threading.Thread(target=child, args=(myshare,))
child1.start()
child2.start()
child1.join()
child2.join()
print(myshare.counter)
print(myshare.histo)
# nowhere it should be different to a count of 1
np.where(myshare.histo != 1)

10
[1 1 1 1 1 1 1 1 1 1]


(array([], dtype=int64),)

Let's' create a much bigger count and check
-----

In [13]:
myshare = Shared(300000)
child1 = threading.Thread(target=child, args=(myshare,))
child2 = threading.Thread(target=child, args=(myshare,))
child1.start()
child2.start()
child1.join()
child2.join()
print(myshare.counter)
# nowhere it should be different to a count of 1
np.where(myshare.histo != 1)

300001


(array([144965, 144966, 144967, ..., 208746, 208747, 208748], dtype=int64),)

Oops! something went wrong!

Let's use a semaphore
----

- Note the use of a semaphore 
- Note that the *child_sem* is careful to release the semaphore when it reaches the *topcount* value

In [14]:
class Shared_SEM:
    def __init__(self, topcount=10):
        self.counter = 0
        self.topcount = topcount
        self.histo = np.zeros(topcount, dtype = np.int32)
        self.onlyme = threading.Semaphore()
        
def child_sem(shared):
    while True:
        shared.onlyme.acquire()
        if shared.counter < shared.topcount:
            shared.histo[shared.counter] += 1
            shared.counter += 1
            shared.onlyme.release()
        else:
            shared.onlyme.release()
            break

In [15]:
share_sem = Shared_SEM(1000000)
child1 = threading.Thread(target=child_sem, args=(share_sem,))
child2 = threading.Thread(target=child_sem, args=(share_sem,))
child1.start()
child2.start()
child1.join()
child2.join()
print(share_sem.counter)
# nowhere it should be different to a count of 1
np.where(share_sem.histo != 1)

1000000


(array([], dtype=int64),)

That is what we were expecting, good!